In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import date
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Import Raw Data
max_age = 30

raw_actress = pd.read_csv('../input/japanese-pornstars-and-adult-videos/actress.csv')
raw_actress.birthday = pd.to_datetime(raw_actress.birthday)
raw_actress['age'] = (pd.Timestamp('2020-02-01') - raw_actress.birthday).astype('timedelta64[Y]')
raw_actress = raw_actress[(raw_actress.age < max_age) | (raw_actress.age.isna())]
raw_actress.head()

In [ ]:
# Prepare Data
actress = raw_actress.copy()
actress.bust = raw_actress.bust.str.replace('cm', '').astype(np.float)
actress.waist = raw_actress.waist.str.replace('cm', '').astype(np.float)
actress.hips = raw_actress.hips.str.replace('cm', '').astype(np.float)
actress.height = raw_actress.height.str.replace('cm', '').astype(np.float)
actress.birthday = pd.to_datetime(raw_actress.birthday)
cup_index = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12, 'M': 13, 'N':14, 'O':15, 'P':16, 'Q':17}
actress['cup_index'] = raw_actress.cup_size.replace(cup_index)

actress = actress.fillna(actress.mean())
actress.head()

# Manually Add Actresses
actress.loc[20000] = [100000, 'EIMI FUKADA', '', '', 158, 'D', 87.8, 63.5, 91.1, '', '', 22, 6]
actress = actress.reset_index()

In [ ]:
measurements = actress[['height', 'bust', 'waist', 'hips', 'age', 'cup_index']]
std_actress = (measurements - measurements.mean()) / measurements.std()
std_actress

In [ ]:
# Calculate Euclidian Distances
distances = pdist(std_actress[['height', 'bust', 'waist', 'hips', 'age', 'cup_index']], metric='euclidean')
dist_matrix = squareform(distances)
df_dist = pd.DataFrame(columns = actress.name, index = actress.name, data = dist_matrix)
df_dist.head()

In [ ]:
# Export Results
df_dist.to_csv('results.csv')